In [0]:
# spark.conf.set("spark.databricks.io.cache.enabled", "true")

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.types import DoubleType, FloatType, LongType, IntegerType
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.window import Window

from constants import (
    BRONZE_LAYER_PATH,
    SILVER_LAYER_PATH,
    ORDER_SILVER_COLUMN_COMMENTS,
    CONSUMER_SILVER_COLUMN_COMMENTS,
    RESTAURANT_SILVER_COLUMN_COMMENTS,
    AB_TEST_REF_SILVER_COLUMN_COMMENTS,
    ABT_SILVER_COLUMN_COMMENTS
)
import utils as u
from schemas import CURRENCY_VALUE_SCHEMA, ITEM_SCHEMA

In [0]:
def add_column_comments(table_name, comments_dict):
    for col_name, comment_text in comments_dict.items():
        safe_comment = comment_text.replace('"', '\\"')
        
        sql_command = f"""
        ALTER TABLE {table_name} ALTER COLUMN {col_name} COMMENT "{safe_comment}"
        """
        spark.sql(sql_command)

def _data_info(df: DataFrame):
    numeric_cols = [f.name for f in df.schema
                    if isinstance(f.dataType, (DoubleType, LongType, IntegerType, FloatType))]

    # total registros
    print(f"Total de registros: {df.count()}")

    # valores nulos
    print("\nValores nulos por coluna:")
    display(df.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in df.columns]))
    
    # dados duplicados
    print("\nNúmero de registros duplicados:")
    display(df.groupBy(df.columns).count().filter(f.col("count") > 1).count())

def _iqr(df: DataFrame, columns: list) -> DataFrame:
    results = []
    for column in columns:
        q1 = df.approxQuantile(column, [0.25], 0.01)[0]
        q3 = df.approxQuantile(column, [0.75], 0.01)[0]
        p95 = df.approxQuantile(column, [0.95], 0.01)[0]
        p99 = df.approxQuantile(column, [0.99], 0.01)[0]
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        results.append((column, q1, q3, iqr, p95, p99, lower_bound, upper_bound))
    
    schema = ["Column", "Q1", "Q3", "IQR", "P95", "P99", "Lower_Bound", "Upper_Bound"]
    return spark.createDataFrame(results, schema)

In [0]:
orders_df = spark.table(f"{BRONZE_LAYER_PATH}.order")
consumers_df = spark.table(f"{BRONZE_LAYER_PATH}.consumer")
restaurants_df = spark.table(f"{BRONZE_LAYER_PATH}.restaurant")
ab_test_df = spark.table(f"{BRONZE_LAYER_PATH}.ab_test_ref")

### Orders


In [0]:
order_id_duplicated_df = orders_df.groupBy(f.col("order_id")).count().filter("count > 1")
print(f"Total de order_id duplicados: {order_id_duplicated_df.count()}")

display(order_id_duplicated_df.select("order_id").limit(10))

In [0]:
order_ids_to_check = [
    row["order_id"]
    for row in order_id_duplicated_df.select("order_id").limit(5).collect()
]

display(orders_df.filter(f.col("order_id").isin(order_ids_to_check)).orderBy("order_id"))

Percebemos uma clara inconsistência no sistema: Mesmos pedidos, com cpf diferentes! Isso parece ser um erro na integridade dos dados.
Poderíamos excluir todos registros com `order_id` iguais, mas vamos manter apenas os que seguirem uma lista de criterios para de-duplicar e deixar apenas `order_id` distintos na tabela. Vamos manter:
1. O mais completo
2. Caso os dois estiverem completos, o com maior valor
3. Caso os dois critérios anteriores sejam iguais entre os registros, trazer apenas o mais recente



In [0]:
orders_df = orders_df.withColumn(
    "is_complete",
    f.when(f.col("items").isNotNull() & (f.col("items") != "[]"), 1).otherwise(0)
)

print(f"Registros antes da deduplicação: {orders_df.count()}")

# criterios para de-duplicar registros com order_id igual
window_spec = Window.partitionBy("order_id").orderBy(
    f.col("is_complete").desc(),
    f.col("order_total_amount").desc(),  
    f.col("order_created_at").desc()
)

orders_df = orders_df.withColumn("row_number", f.row_number().over(window_spec))
orders_df = orders_df.filter(f.col("row_number") == 1).drop("row_number", "is_complete")

print(f"Registros após de-duplicação: {orders_df.count()}")

Diminuição considerável do total de registros

In [0]:
month_df = orders_df.withColumn(
    "year_month",
    f.date_format(f.col("order_created_at"), "yyyy-MM")
)

order_by_month_df = (
    month_df
    .groupBy(f.col("year_month"))
    .count()
    .orderBy(f.col("year_month"))
)

print("Total de pedidos por mês")
display(order_by_month_df)

Após a de-duplicação só ficaram pedidos para jan/19

In [0]:
orders_df = u.json_column_parse(orders_df, "items", ITEM_SCHEMA)
orders_df.printSchema()

In [0]:
orders_df.filter(f.col("items_parsed").isNull()).count()

In [0]:
item_parsed_null_df = orders_df.filter(
    f.col("items_parsed").isNull() &
    (f.col("items").isNotNull()) &
    (f.col("items") != "[]")
)

print("Amostra de registros onde o parse falhou (items_parsed é nulo):")
display(item_parsed_null_df.select("order_id", "items"))

In [0]:
initial_value_double = f.lit(0).cast(DoubleType())

orders_df = (
    orders_df.withColumn(
        "total_items_quantity",
        f.aggregate(
            "items_parsed",
            initial_value_double,
            lambda acc, item: acc
            + f.coalesce(item["quantity"].cast(DoubleType()), initial_value_double),
        ),
    )
    .withColumn(
        "total_items_value",
        f.aggregate(
            "items_parsed",
            initial_value_double,
            lambda acc, item: acc
            + f.coalesce(
                f.round(item["totalValue"]["value"].cast(DoubleType()) / 100, 2),
                initial_value_double,
            )
            + f.aggregate(
                item["garnishItems"],
                initial_value_double,
                lambda garnish_acc, garnish_item: garnish_acc
                + f.coalesce(
                    (garnish_item["totalValue"]["value"].cast(DoubleType()) / 100),
                    initial_value_double,
                ),
            ),
        ),
    )
    .withColumn(
        "total_items_addition",
        f.aggregate(
            "items_parsed",
            initial_value_double,
            lambda acc, item: acc
            + f.coalesce(
                f.round(item["totalAddition"]["value"].cast(DoubleType()) / 100, 2),
                initial_value_double,
            ),
        ),
    )
    .withColumn(
        "total_items_discount",
        f.aggregate(
            "items_parsed",
            initial_value_double,
            lambda acc, item: acc
            + f.coalesce(
                f.round(item["totalDiscount"]["value"].cast(DoubleType()) / 100, 2),
                initial_value_double,
            ),
        ),
    )
    .withColumn("num_distinct_items", f.size(f.col("items_parsed")))
)

orders_df.printSchema()

In [0]:
# orders
_data_info(orders_df)

**ATENÇÃO:** Durante a fase de qualidade de dados, foram identificados 5.559 pedidos (~0.23% do total) sem um `customer_id` associado. Como este é o identificador único do cliente, essencial para o teste A/B e análise de retenção, esses registros serão removidos para garantir a integridade e precisão da análise subsequente. Além disso, não foi possível preenchê-lo a partir de outras informações como `customer_name`. Por fim, serão removidos os outros 2 registros que tiveram o items_parsed nulo. Dado o volume, removê-los não trará nenhum tipo de viés significativo a análise.

Todos os registros de `order` estão sem `delivery_address_latitude`, `delivery_address_longitude`, 
`merchant_latitude` e `merchant_longitude`. Essas colunas serão deletadas.

In [0]:
orders_cleaned_df = orders_df.na.drop(subset=["customer_id", "items_parsed"])
orders_cleaned_df = orders_cleaned_df.drop(
        f.col("delivery_address_latitude"),
        f.col("delivery_address_longitude"),
        f.col("merchant_latitude"),
        f.col("merchant_longitude"),
)

_data_info(orders_cleaned_df)

In [0]:
orders_cleaned_df = orders_cleaned_df.drop(
        f.col("delivery_address_latitude"),
        f.col("delivery_address_longitude"),
        f.col("merchant_latitude"),
        f.col("merchant_longitude"),
)

orders_cleaned_df.columns

In [0]:
print("Verificando a relação entre o agendamento e a data de agendamento:")
display(
    orders_cleaned_df.groupBy(f.col("order_scheduled")).agg(
        f.count("*").alias("total_orders"),
        f.count(f.when(f.col("order_scheduled_date").isNull(), 1)).alias("order_scheduled_date null")
    )
)

Pela análise, chegamos a conclusão que:
1. **99,98%** dos pedidos são feitos para entrega imediata
2. As colunas `order_scheduled` e `order_scheduled_date` estão consistentes entre si

In [0]:
order_scheduled_month_df = orders_cleaned_df.withColumn(
    "year_month",
    f.date_format(f.col("order_scheduled_date"), "yyyy-MM")
)

order_scheduled_by_month_df = (
    order_scheduled_month_df
    .groupBy(f.col("year_month"))
    .count()
    .orderBy(f.col("year_month"))
)

print("Total de pedidos agendados por mês")
display(order_scheduled_by_month_df)

Além do pouco preenchimento da coluna `order_scheduled_date`, ela possui apenas datas do ano de 2016, onde na realidade deveria ser 2019. Sendo assim, vamos dropar essa coluna também.

In [0]:
orders_cleaned_df = orders_cleaned_df.drop(f.col("order_scheduled_date"))
orders_cleaned_df.columns

In [0]:
numeric_cols = [
    "order_total_amount",
    "total_items_value",
    "total_items_quantity",
    "num_distinct_items"
]

print("Estatísticas Descritivas")
display(orders_cleaned_df.describe(numeric_cols).filter(f.col("summary") != "count"))

Aqui eu percebemos que estavamos ignorando itens de guarnição e com isso, alguns pedidos possuiam valor > 0 e valor total dos itens = 0. Voltamos e adicionamos, agora não existe mais esse caso.

In [0]:
display(
    orders_cleaned_df.filter(
        (f.col("order_total_amount") > 0) & (f.col("total_items_value") == 0)
    )
)

#### Estatísticas descritivas das colunas numéricas
##### Análise dos valores dos pedidos

* Valor Bruto vs. Valor Final: A observação mais importante é que a média do valor bruto dos itens (`total_items_value`) é de **R$ 56,45**, enquanto a média do valor total pago (`order_total_amount`) é de **R$ 47,85**. Isso indica que, em média, há um impacto de **R$ 8,60** por pedido proveniente de descontos (sejam eles dos restaurantes ou de campanhas), que reduzem o valor final pago pelo cliente.

* Extrema Assimetria e Outliers: Para ambas as colunas de valor, o desvio padrão (**R$ 101-112**) é mais que o dobro da média. Isso, somado ao valor máximo de **R$ 138.750,90**, confirma que a distribuição dos dados é extremamente assimétrica à direita. A grande maioria dos pedidos são de baixo valor, mas um pequeno número de pedidos de valor altíssimo distorce fortemente a média.

##### Análise das quantidades na cesta
* Comportamento Típico de Compra: O comportamento padrão do cliente é fazer pedidos pequenos e focados. Em média, um pedido contém **2.34** itens no total (`total_items_quantity`) e **2.03** itens distintos (`num_distinct_items`). Isso sugere que os usuários geralmente não compram múltiplos do mesmo item na mesma cesta.

* Outliers de Quantidade: Assim como nos valores, os valores máximos de **2.306** itens totais e **100** itens distintos são outliers extremos e não refletem o comportamento de um cliente comum.

##### Conclusão e Próximos Passos
A análise estatística revela um cenário claro: o volume principal de pedidos é gerado por compras pequenas e de baixo valor. A presença de outliers extremos em todas as métricas numéricas é o ponto de maior atenção.

In [0]:
iqr_df = _iqr(orders_cleaned_df, numeric_cols)
display(iqr_df)

In [0]:
p95 = iqr_df.select(f.col("P95")).where(f.col("Column") == "order_total_amount").collect()[0][0]
print(f"P95: {p95}")

orders_cleaned_df.filter(
    (f.col("order_total_amount") > p95)).count()

A distribuição de order_total_amount tem uma cauda longa a direita. Porém, 95% dos pedidos tem valores até R$ 105,00. Como existem 132.110 pedidos com valor maior que o p95, vamos utilizar uma técnica de capping para não perder informação e remover o viés. Sendo assim, para o propósito de avaliar o impacto médio desta campanha, estamos considerando que o comportamento de um cliente que gasta R$ 110 é similar ao de um cliente que gasta R$ 10.000. Ambos são 'clientes de alto valor', não queremos que a diferença enorme entre eles distorça a média de todo o grupo.

In [0]:
orders_final_df = orders_cleaned_df.withColumn(
    "order_total_amount_capped",
    f.when(f.col("order_total_amount") > p95, p95)
     .otherwise(f.col("order_total_amount"))
)

display(orders_final_df.limit(5))

In [0]:
silver_order_table_name = f"{SILVER_LAYER_PATH}.order"

spark.sql(f"DROP TABLE IF EXISTS {silver_order_table_name}")

orders_final_df = orders_final_df.withColumn("partition_date", f.to_date(f.col("order_created_at")))

(orders_final_df.write
 .partitionBy("partition_date")
 .format("delta")
 .mode("overwrite")
 .saveAsTable(silver_order_table_name))

# adicionando descrição das colunas
add_column_comments(silver_order_table_name, ORDER_SILVER_COLUMN_COMMENTS)

print("Tabela 'order' da camada Silver criada com sucesso!")

### Consumer

In [0]:
consumers_df.printSchema()

In [0]:
_data_info(consumers_df)

In [0]:
customer_id_duplicated_df = consumers_df.groupBy(f.col("customer_id")).count().filter(f.col("count") > 1)
print(f"Total de customer_id duplicados: {customer_id_duplicated_df.count()}")

O tempo que um cliente está na plataforma é uma boa variavel para prever comportamento. Clientes novos podem reagir a cupons de forma diferente de clientes antigos.

**Hipótese:** 
O cupom é mais eficaz para converter novos usuários do que para reter usuários antigos.

**Ação:** 
Calcular o número de dias entre a data de criação da conta (`created_at`) e o início do nosso período de análise.

In [0]:
min_created_at_range = consumers_df.select(f.to_date(f.min(f.col("created_at")))).collect()[0][0]
max_created_at_range = consumers_df.select(f.to_date(f.max(f.col("created_at")))).collect()[0][0]

print(f"O cliente mais antigo entrou na plataforma em: {min_created_at_range.strftime('%d/%m/%Y')}")
print(f"O cliente mais novo entrou na plataforma em: {max_created_at_range.strftime('%d/%m/%Y')}")

In [0]:
campaign_start_date = f.to_date(f.lit("2019-01-01"))

consumers_df = consumers_df.withColumn(
  "days_since_signup",
  f.date_diff(campaign_start_date, f.to_date(f.col("created_at")))  
)

# clientes com menos de 90 dias até a campanha consideramos novos clientes
consumers_df = consumers_df.withColumn(
    "maturity_segment",
    f.when(f.col("days_since_signup") < 90, "novo")
     .otherwise("antigo")
)

print("Segmentação por maturidade do cliente")
display(consumers_df.groupBy("maturity_segment").count())

Como não temos novos clientes de acordo com o periodo estabelecido, podemos remover a coluna maturity_segment

In [0]:
display(consumers_df.groupBy("active").count())

consumers_final_df = consumers_df.drop(f.col("maturity_segment"))

Um cupom pode ter o objetivo de reativar clientes inativos ou simplesmente aumentar a frequência dos que já são ativos.

**Hipótese:** O cupom é mais eficaz para reativar clientes inativos do que para aumentar o gasto de clientes já ativos.

**Ação:** Usar a coluna `active` diretamente na nossa análise final para comparar os resultados do teste A/B entre esses dois grupos.

Isso permitirá responder à pergunta **"A campanha teve impacto?"** de forma mais granular. Em vez de uma única resposta:

"No geral, a campanha teve um aumento de X no ticket médio."

"Porém, este efeito foi impulsionado principalmente pelos Clientes Novos, que aumentaram seu gasto em Y, enquanto nos Clientes Antigos o efeito foi insignificante."

"Além disso, observamos que o cupom foi muito eficaz para reativar clientes inativos, aumentando a probabilidade de compra em Z%, mas teve pouco efeito na frequência de compra dos clientes já ativos."

In [0]:
display(consumers_final_df.groupBy("language").count())

A coluna `language` não traz informação relevante e pode ser deletada. Além disso, as colunas
`created_at`, `customer_name`, `customer_phone_area` e `customer_phone_number` para a analise que estamos realizando não são uteis, pois o nome não é um identificador único, o código de área poderia ser usado para avaliar a localização, mas isso já está presente em orders e o número de telefone é um dado PII que não tem nenhuma relevância para o case. Sendo assim, também serão excluídas.

In [0]:
consumers_final_df = consumers_final_df.drop(
    f.col("language"),
    f.col("created_at"), 
    f.col("customer_name"), 
    f.col("customer_phone_area"), 
    f.col("customer_phone_number")
)

consumers_final_df.printSchema()

In [0]:
silver_consumer_table_name = f"{SILVER_LAYER_PATH}.consumer"

spark.sql(f"DROP TABLE IF EXISTS {silver_consumer_table_name}")

(consumers_final_df.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable(silver_consumer_table_name))

# adicionando descrição das colunas
add_column_comments(silver_consumer_table_name, CONSUMER_SILVER_COLUMN_COMMENTS)

print("Tabela 'consumer' da camada Silver criada com sucesso!")

### Restaurant

In [0]:
restaurants_df.printSchema()

Primeiro passo é renomear a coluna `id` para `merchant_id`, para ficar consistente com a tabela order.

In [0]:
restaurants_df = restaurants_df.withColumnRenamed("id", "merchant_id")
restaurants_df.printSchema()

In [0]:
merchant_id_duplicated_df = restaurants_df.groupBy(f.col("merchant_id")).count().filter(f.col("count") > 1)
print(f"Total de merchant_id duplicados: {merchant_id_duplicated_df.count()}")

In [0]:
_data_info(restaurants_df)

In [0]:
mean_min_order_value = restaurants_df.select(f.mean(f.col("minimum_order_value"))).first()[0]
print(f"A média para 'minimum_order_value' é: {mean_min_order_value:.2f}")

restaurants_df = restaurants_df.na.fill(mean_min_order_value, subset=["minimum_order_value"])

In [0]:
numeric_cols = [
    "price_range",
    "average_ticket",
    "delivery_time",
    "minimum_order_value"
]

print("Estatísticas Descritivas")
display(restaurants_df.describe(numeric_cols).filter(f.col("summary") != "count"))

#### Estatísticas descritivas das colunas numéricas
##### Análise dos valores

`price_range`

* A média é de **2.56** em uma escala que vai de 1 a 5. 
* Isso indica que a maioria dos restaurantes na plataforma se concentra em uma faixa de preço intermediária. Não há um domínio nem de restaurantes muito baratos (próximo de 1) nem muito caros (próximo de 5). A distribuição é relativamente centrada.

`average_ticket`

* A média do ticket por restaurante é de **R$ 53,72**, com um desvio padrão de **R$ 19,56**. 
* Há uma variabilidade moderada no ticket médio entre os restaurantes. O desvio padrão não é excessivamente alto em comparação com a média, sugerindo que, embora existam restaurantes com tickets mais altos ou mais baixos, a maioria orbita em torno da faixa de **R$ 34 a R$ 73** (aproximadamente um desvio padrão da média).

`delivery_time`

* A média do tempo de entrega padrão é de apenas **3.18** (provavelmente em minutos, mas a escala parece estranha ou os dados podem estar normalizados), mas o desvio padrão é alto (**8.42**) e o máximo é **60**. 

* A média muito baixa e o desvio padrão alto indicam que esta coluna pode ter problemas de qualidade ou uma unidade de medida que não está clara. A maioria dos restaurantes talvez tenha um valor padrão baixo no sistema, enquanto alguns têm tempos de entrega mais realistas. Para os objetivos da análise, esta coluna tem baixa confiabilidade e provavelmente não será útil. 

* O objetivo central do case é medir o impacto de um cupom na retenção e no comportamento de compra dos usuários, acreditamos que o tempo de entrega padrão pode não influenciar na eficacia do cupom.

`minimum_order_value`

* A média do valor mínimo para um pedido é de **R$ 28,03**. O mínimo é **R$ 0** e o máximo é **R$ 285**. 

* A média de **R$ 28,00** é um valor razoável para um pedido mínimo. O desvio padrão de **R$ 25,24** e o máximo de **R$ 285,00** indicam, novamente, a presença de outliers.

In [0]:
restaurants_cleaned_df = restaurants_df.drop(f.col("delivery_time"))
restaurants_cleaned_df.printSchema()

In [0]:
mean_avg_ticket = restaurants_df.select(f.round(f.mean(f.col("average_ticket")), 2)).first()[0]
mean_min_order_value = restaurants_df.select(f.round(f.mean(f.col("minimum_order_value")), 2)).first()[0]

restaurants_cleaned_df = restaurants_cleaned_df.withColumn(
    "restaurant_profile",
    f.when(
        (f.col("average_ticket") >= mean_avg_ticket) & (f.col("minimum_order_value") >= mean_min_order_value),
        "Perfil Alto Valor"
    ).when(
        (f.col("average_ticket") < mean_avg_ticket) & (f.col("minimum_order_value") < mean_min_order_value),
        "Perfil Baixo Valor"
    ).otherwise("Perfil Intermediário")
)

**As seguintes colunas serão descartadas**

`created_at`

* A data de criação do restaurante é uma informação sobre a "maturidade" do parceiro. Embora interessante, o impacto do cupom está muito mais ligado à maturidade do clientebdo que à do restaurante. Manter essa coluna adicionaria uma complexidade que não é central para o problema.

`enabled`

* Esta flag indica se o restaurante está ativo. Como a nossa análise é baseada em pedidos que já ocorreram, podemos assumir que todos os restaurantes envolvidos nesses pedidos estavam habilitados no momento da transação. Portanto, esta coluna não nos ajudará a segmentar ou a entender o resultado do teste A/B. As duas celulas subsequentes mostram que de fato existem pedidos em restaurantes que estão marcados como inativos.

`merchant_zip_code`

* Análise: O CEP é uma informação geográfica muito granular. Para uma análise de negócio de alto nível, agrupar por estado ou cidade é muito mais útil e informativo. O CEP não oferece um nível de agregação prático para este case.

In [0]:
display(restaurants_cleaned_df.groupBy(f.col("enabled")).count())

In [0]:
inactive_restaurants_df = restaurants_cleaned_df.filter(f.col("enabled") == False).select("merchant_id")

inactive_orders_df = orders_cleaned_df.join(
    inactive_restaurants_df.hint("broadcast"),
    "merchant_id",
    "inner"
)

print(f"Total de pedidos realizados em parceiros inativos: {inactive_orders_df.count()}")

display(inactive_orders_df.limit(5))

Foi verificado que 728174 pedidos foram realizados em restaurantes que, na base de parceiros, constam como inativos. Isso pode indicar uma latência na atualização dos dados. Para esta análise, os pedidos foram mantidos, mas em um ambiente de produção, a causa raiz dessa inconsistência deveria ser investigada.

In [0]:
restaurants_final_df = restaurants_cleaned_df.drop(
    f.col("enabled"), 
    f.col("created_at"), 
    f.col("merchant_zip_code")
)

restaurants_final_df.printSchema()

In [0]:
silver_restaurant_table_name = f"{SILVER_LAYER_PATH}.restaurant"

spark.sql(f"DROP TABLE IF EXISTS {silver_restaurant_table_name}")

(restaurants_final_df.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable(silver_restaurant_table_name))

# adicionando descrição das colunas
add_column_comments(silver_restaurant_table_name, RESTAURANT_SILVER_COLUMN_COMMENTS)

print("Tabela 'restaurant' da camada Silver criada com sucesso!")

### A/B Test Ref

In [0]:
ab_test_df.printSchema()

In [0]:
_data_info(ab_test_df)

In [0]:
ab_test_duplicated_df = ab_test_df.groupBy(f.col("customer_id")).count().filter(f.col("count") > 1)
print(f"Total de customer_id duplicados em ab_ref_test: {ab_test_duplicated_df.count()}")

In [0]:
display(ab_test_df.groupBy(f.col("is_target")).count())

Aqui, percebemos que o ab_test_ref tem mais usuários distintos que o consumers. É uma descoberta, mas não é um bloqueio. Isso pode acontecer por vários motivos em um ambiente de produção, como um erro de ingestão ou latência na replicação de dados.

In [0]:
missing_customers = ab_test_df.join(
    consumers_df,
    "customer_id",
    "left_anti"
).count()

print(f"O DataFrame ab_test tem {missing_customers} usuários a mais que o consumers_df.")

Essa informação é importante para determinar a ordem dos joins das tabelas e não perder informação. 

orders INNER JOIN ab_test_ref LEFT JOIN consumers LEFT JOIN restaurants

In [0]:
ab_test_final_df = ab_test_df.withColumn(
    "is_target", 
    (f.col("is_target") == "target")
)

ab_test_final_df.printSchema()

In [0]:
silver_ab_test_ref_table_name = f"{SILVER_LAYER_PATH}.ab_test_ref"

spark.sql(f"DROP TABLE IF EXISTS {silver_ab_test_ref_table_name}")

(ab_test_final_df.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable(silver_ab_test_ref_table_name))

# adicionando descrição das colunas
add_column_comments(silver_ab_test_ref_table_name, AB_TEST_REF_SILVER_COLUMN_COMMENTS)

print("Tabela 'ab_test_ref' da camada Silver criada com sucesso!")

### Join

Aqui poderíamos usar os DataFrames já criados, porém para não ser preciso rodar todas as transformações novamente, vamos recuperar as tabelas aqui permitindo rodar o notebook a partir dessa célula para chegar na tabela de análise completa.

In [0]:
orders_df = spark.table(f"{SILVER_LAYER_PATH}.order")
consumers_df = spark.table(f"{SILVER_LAYER_PATH}.consumer")
restaurants_df = spark.table(f"{SILVER_LAYER_PATH}.restaurant")
ab_test_df = spark.table(f"{SILVER_LAYER_PATH}.ab_test_ref") 

In [0]:
base_df = orders_df.join(
    ab_test_df,
    "customer_id",
    "inner"
)

print(f"Total de registros após INNER JOIN com ab_test_ref: {base_df.count()}")

A manutenção de todos os registros indica que da base gerada de order, todos os pedidos foram realizados por pessoas participantes do teste. 

In [0]:
base_df = base_df.join(
  consumers_df,  
  "customer_id",
  "left"
)

In [0]:
abt_final_df = base_df.join(
  restaurants_df,
  "merchant_id",
  "left"
)

base_df.printSchema()

In [0]:
silver_abt_final_table_name = f"{SILVER_LAYER_PATH}.abt_final"

spark.sql(f"DROP TABLE IF EXISTS {silver_abt_final_table_name}")

(abt_final_df.write
 .partitionBy("partition_date")
 .format("delta")
 .mode("overwrite")
 .saveAsTable(silver_abt_final_table_name))

# adicionando descrição das colunas
add_column_comments(silver_abt_final_table_name, ABT_SILVER_COLUMN_COMMENTS)

print("Tabela 'abt_final' da camada Silver criada com sucesso!")